In [2]:
import pandas as pd
import pickle
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import NBEATSModel
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from datetime import datetime, timedelta
import numpy as np
import torch

/opt/anaconda3/envs/lstm-example/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [4]:
# Load the preprocessed data
data_path = 'prepared_data_for_prophet.csv'
df = pd.read_csv(data_path)

# Convert 'ds' column to datetime
df['ds'] = pd.to_datetime(df['ds'])

# Load the model using Darts
try:
    nbeats_model = NBEATSModel.load('/Users/ammarahmad/Documents/Its IT Group/Fuel Price TimeSeries/lstm-example/NBEATSModel/trained_nbeats_model.pkl')
    print("Model loaded successfully.")
except Exception as e:
    print("Failed to load model:", e)

Failed to load model: Invalid magic number; corrupt file?


/opt/anaconda3/envs/lstm-example/lib/python3.10/site-packages/darts/models/forecasting/torch_forecasting_model.py:1747: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model: 

In [13]:
def check_NaN(data):
    # Select only numeric columns for spline interpolation
    numeric_columns = data.select_dtypes(include=[np.number]).columns

    print("Number of NaN values:\n", data[numeric_columns].isna().sum())

In [14]:
# Ensure the 'ds' column is in datetime format
df['ds'] = pd.to_datetime(df['ds'])

# Define the start and end date for the prediction
start_date = datetime(2024, 7, 1, 0, 0)
end_date = start_date + timedelta(weeks=3)  # 3 weeks

# Define the period for the past data (3 months before the start date)
past_start_date = start_date - timedelta(weeks=12)  # 3 months ~ 12 weeks

# Filter the past data (past 3 months)
past_data = df[(df['ds'] >= past_start_date) & (df['ds'] < start_date)]

# Extract target series ('y') and regressors
regressors = [col for col in df.columns if col not in ['y', 'ds']]

In [15]:
# Convert past data to TimeSeries format for Darts
series = TimeSeries.from_dataframe(past_data, 'ds', 'y')
covariates = TimeSeries.from_dataframe(past_data, 'ds', regressors)

In [16]:
# Scale the data
scaler_series = Scaler()
scaler_covariates = Scaler()

series = scaler_series.fit_transform(series)
covariates = scaler_covariates.fit_transform(covariates)

In [17]:
# Number of data points to predict (3 weeks of hourly data)
n = 24 * 7 * 3

# Use the model to predict future values
predicted_series = nbeats_model.predict(n=n, series=series, past_covariates=covariates)

# Convert predicted time series back to a DataFrame for easier visualization and comparison
predicted_df = predicted_series.pd_dataframe()
predicted_df.columns = ['predicted']

# Print the first few rows of the prediction results
print(predicted_df.head())

AttributeError: 'NoneType' object has no attribute 'set_predict_parameters'